# rent curve 拟合

## 计算city/county乡村土地地租均值

In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

In [ ]:


def compute_avg_rural_rent(prov, city, agr_path, admin_path, builtup_path):
    """
    Compute the average agricultural land rent for a given city,
    restricted to areas within the administrative boundary and outside the built-up area.
    
    Parameters:
        prov: Province name (e.g., 'Hebei')
        city: City or county name (e.g., 'Baoding' or 'Anxin County')
        agr_path: Path to agricultural land value CSV file
        admin_path: Path to China's administrative boundary shapefile
        builtup_path: Path to built-up area shapefile
    
    Returns:
        Average price_m2 (float), or None if no matching data is found
    """

    df = pd.read_csv(agr_path)
    gdf_agr = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
    )


    gdf_admin = gpd.read_file(admin_path).to_crs("EPSG:4326")
    gdf_admin.loc[:, "geometry"] = gdf_admin.geometry.buffer(0)
    gdf_admin = gdf_admin.rename(
        columns={
            "NAME_1": "province",
            "NAME_2": "city",
            "NAME_3": "county",
        }
    )


    gdf_builtup = gpd.read_file(builtup_path).to_crs("EPSG:4326")
    gdf_builtup.loc[:, "geometry"] = gdf_builtup.geometry.buffer(0)

    mask = (
        gdf_admin.province.str.lower().str.strip() == prov.lower().strip()
    )
    city_mask = (
        gdf_admin.city.str.lower().str.strip() == city.lower().strip()
    )
    county_mask = (
        gdf_admin.county.str.lower().str.strip() == city.lower().strip()
    )
    gdf_sub = gdf_admin.loc[mask & (city_mask | county_mask)]
    if gdf_sub.empty:
        print(f"cannot find {prov}-{city} in the administrative boundary shapefile.")
        return None


    admin_union = gdf_sub.geometry.union_all()
    gdf_builtup_sub = gdf_builtup.loc[
        gdf_builtup.geometry.intersects(admin_union)
    ]
    builtup_union = (
        gdf_builtup_sub.geometry.union_all()
        if not gdf_builtup_sub.empty else None
    )


    in_admin = gdf_agr.geometry.within(admin_union)
    if builtup_union is not None:
        in_rural = ~gdf_agr.geometry.within(builtup_union)
        gdf_final = gdf_agr.loc[in_admin & in_rural]
    else:
        gdf_final = gdf_agr.loc[in_admin]

    if gdf_final.empty:
        print(f"{prov}-{city} has no valid agricultural land points.")
        return None
    
    # consider the present discounted value of future income flows from land use.
    # discount rate is 5% per year
    return gdf_final["price_m2"].mean()/0.05


In [7]:
def parse_prov_city_from_filename(filename: str):
    """
    Parse province and city from filename.
    Example: 'Anhui-Anqing_hp.csv' → ('Anhui', 'Anqing')
    """
    base = os.path.basename(filename)
    if base.endswith('_hp.csv'):
        base = base[:-len('_hp.csv')]
    try:
        prov, city = base.split('-', 1)
        return prov.strip(), city.strip()
    except ValueError:
        raise ValueError(f"Failed to parse province and city from filename: {filename}")


In [8]:
file_name = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/City_hp/Anhui-Anqing_hp.csv"
prov, city = parse_prov_city_from_filename(file_name)
agr_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/agr_val_2022.csv"
admin_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Raw/China_Adm_2020/China2020County.shp"
builtup_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/China_BuiltUp_300kCities_2020/China_BuiltUp_300kCities_2020.shp"

gdf_adimin_county = gpd.read_file(admin_path)
gdf_adimin_county.columns



Index(['地名', '区划码', '县级', '县级码', '县级类', '地级', '地级码', '地级类', '省级', '省级码', '省级类',
       '曾用名', '备注', 'ENG_NAME', 'VAR_NAME', 'code', 'NAME_3', 'VAR_NAME3',
       'GID_3', 'TYPE_3', 'NAME_2', 'VAR_NAME2', 'GID_2', 'TYPE_2', 'NAME_1',
       'VAR_NAME1', 'GID_1', 'TYPE_1', 'year', 'geometry'],
      dtype='object')

In [ ]:
avg_price = compute_avg_rural_rent(prov, city, agr_path, admin_path, builtup_path) 
print(f"average algricultural land rent in {prov}-{city} is {avg_price} CNY/m2")

/opt/anaconda3/envs/midway-debug/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: /Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/China_BuiltUp_300kCities_2020/China_BuiltUp_300kCities_2020.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


average algricultural land rent in Anhui-Anqing is 29.408967034675488 CNY/m2



### Method: Converting Annual Agricultural Land Output into Land Value for Comparison with Housing Prices

This section explains how to convert annual agricultural land productivity (in RMB per square meter per year) into an estimate of land value that can be meaningfully compared to urban land prices or housing prices. The conversion relies on standard economic principles of income capitalization.

#### Step 1: Starting Point – Annual Output

The computed value:

- `R` denotes the **annual agricultural output per square meter**, in RMB/m²/year.

This value reflects the flow-based productivity of the land, based on agro-climatic yield estimates from GAEZ and converted into 2022 RMB.

#### Step 2: Capitalization into Land Value

According to standard urban economics and land valuation theory (e.g., Alonso-Muth-Mills framework), the land value `P` is the present discounted value of future income flows from land use.

For perpetual use rights, land value is computed as:

```math
P = \frac{R}{r}
````

Where:

* `R` is the annual land return (e.g., 5 RMB/m²/year)
* `r` is the discount rate (e.g., 5%)

This represents a standard perpetuity formula in asset pricing.

#### Step 3: Finite Land Use Horizon

For cases where land use is time-limited (e.g., 30-year rural land contracts), use the finite horizon annuity formula:

```math
P = R \cdot \frac{1 - (1 + r)^{-T}}{r}
```

Where:

* `T` is the land use duration (e.g., 30 years)
* `r` is the annual discount rate

For example, with `R = 5`, `r = 0.05`, and `T = 30`:

```math
P \approx 5 \cdot \frac{1 - (1.05)^{-30}}{0.05} \approx 5 \cdot 15.37 = 76.85 \ \text{RMB/m²}
```

This result is lower than the perpetual case `P = R / r = 100` due to the finite time horizon.

#### Step 4: Recommended Parameters

| Assumption        | Suggested Value | Interpretation                                               |
| ----------------- | --------------- | ------------------------------------------------------------ |
| Discount rate `r` | 3% – 7%         | Based on social discount rate or opportunity cost of capital |
| Time horizon `T`  | 30 years        | Typical rural land contract period in China                  |



#### Conclusion

The resulting `land_price` column gives an estimate of the capitalized value of agricultural land per square meter, expressed in 2022 RMB. This value is directly comparable to urban residential land prices or housing floor prices (excluding construction cost), enabling integrated spatial comparisons of land use efficiency.


In [ ]:
def compute_avg_rural_rent_fast(prov, city, agr_path, admin_path, builtup_path):
    import pandas as pd
    import geopandas as gpd

    # 读取农业点 + 空间转换
    df = pd.read_csv(agr_path)
    gdf_agr = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326")

    # 读取行政区
    gdf_admin = gpd.read_file(admin_path).to_crs("EPSG:4326")
    gdf_admin = gdf_admin.rename(columns={"NAME_1": "province", "NAME_2": "city", "NAME_3": "county"})
    gdf_admin["geometry"] = gdf_admin.geometry.buffer(0)

    # 筛选省+市或县
    mask_prov = gdf_admin["province"].str.strip().str.lower() == prov.strip().lower()
    mask_city = gdf_admin["city"].str.strip().str.lower() == city.strip().lower()
    mask_county = gdf_admin["county"].str.strip().str.lower() == city.strip().lower()
    gdf_sub = gdf_admin[mask_prov & (mask_city | mask_county)]
    if gdf_sub.empty:
        print(f"❌ cannot find {prov}-{city}")
        return None
    admin_union = gdf_sub.geometry.unary_union

    # ❗提前裁剪点
    minx, miny, maxx, maxy = admin_union.bounds
    gdf_agr_crop = gdf_agr.cx[minx:maxx, miny:maxy]

    # 读取建成区
    gdf_builtup = gpd.read_file(builtup_path).to_crs("EPSG:4326")
    gdf_builtup["geometry"] = gdf_builtup.geometry.buffer(0)
    gdf_builtup_sub = gdf_builtup[gdf_builtup.geometry.intersects(admin_union)]
    builtup_union = gdf_builtup_sub.geometry.unary_union if not gdf_builtup_sub.empty else None

    # 快速空间判断
    in_admin = gdf_agr_crop.geometry.within(admin_union)
    in_builtup = gdf_agr_crop.geometry.within(builtup_union) if builtup_union else pd.Series(False, index=gdf_agr_crop.index)

    gdf_final = gdf_agr_crop[in_admin & ~in_builtup]

    if gdf_final.empty:
        print(f"{prov}-{city} 有效点为空")
        return None

    return gdf_final["price_m2"].mean() / 0.05


In [ ]:
avg_price = compute_avg_rural_rent_fast(prov, city, agr_path, admin_path, builtup_path) 
print(f"average algricultural land rent in {prov}-{city} is {avg_price} CNY/m2")

/var/folders/4g/6_8lhyp147394q93651p5kyw0000gn/T/ipykernel_78584/1784599200.py:22: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  admin_union = gdf_sub.geometry.unary_union
/opt/anaconda3/envs/midway-debug/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: /Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/China_BuiltUp_300kCities_2020/China_BuiltUp_300kCities_2020.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
/var/folders/4g/6_8lhyp147394q93651p5kyw0000gn/T/ipykernel_78584/1784599200.py:32: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  builtup_union = gdf_builtup_sub.geometry.unary_union if not gdf_builtup_sub.empty else None


average algricultural land rent in Anhui-Anqing is 29.408967034675488 CNY/m2


In [ ]:
def compute_avg_rural_rent_fast2(prov, city, agr_path, admin_path, builtup_path):
    import pandas as pd
    import geopandas as gpd

    df = pd.read_csv(agr_path)
    gdf_agr = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326")

    gdf_admin = gpd.read_file(admin_path).to_crs("EPSG:4326")
    gdf_admin = gdf_admin.rename(columns={"NAME_1": "province", "NAME_2": "city", "NAME_3": "county"})

    # 修复无效geometry仅在必要时
    if not gdf_admin.is_valid.all():
        gdf_admin["geometry"] = gdf_admin.buffer(0)

    mask_prov = gdf_admin["province"].str.strip().str.lower() == prov.strip().lower()
    mask_city = gdf_admin["city"].str.strip().str.lower() == city.strip().lower()
    mask_county = gdf_admin["county"].str.strip().str.lower() == city.strip().lower()
    gdf_sub = gdf_admin[mask_prov & (mask_city | mask_county)]
    if gdf_sub.empty:
        print(f"❌ cannot find {prov}-{city}")
        return None

    admin_union = gdf_sub.geometry.unary_union
    minx, miny, maxx, maxy = admin_union.bounds
    gdf_agr_crop = gdf_agr.cx[minx:maxx, miny:maxy]

    # 行政区空间过滤（sjoin）
    gdf_admin_poly = gpd.GeoDataFrame(geometry=[admin_union], crs="EPSG:4326")
    gdf_in_admin = gpd.sjoin(gdf_agr_crop, gdf_admin_poly, predicate="within", how="inner")

    # 读取建成区并筛选
    gdf_builtup = gpd.read_file(builtup_path).to_crs("EPSG:4326")
    if not gdf_builtup.is_valid.all():
        gdf_builtup["geometry"] = gdf_builtup.buffer(0)
    gdf_builtup_sub = gdf_builtup[gdf_builtup.geometry.intersects(admin_union)]

    # 从 admin 点中排除建成区点
    if not gdf_builtup_sub.empty:
        gdf_not_in_builtup = gpd.sjoin(gdf_in_admin, gdf_builtup_sub, predicate="within", how="left", lsuffix="admin", rsuffix="builtup")
        gdf_final = gdf_not_in_builtup[gdf_not_in_builtup["index_builtup"].isna()]
    else:
        gdf_final = gdf_in_admin

    if gdf_final.empty:
        print(f"{prov}-{city} 有效点为空")
        return None

    return gdf_final["price_m2"].mean() / 0.05


In [ ]:
avg_price = compute_avg_rural_rent_fast2(prov, city, agr_path, admin_path, builtup_path) 
print(f"average algricultural land rent in {prov}-{city} is {avg_price} CNY/m2")

/var/folders/4g/6_8lhyp147394q93651p5kyw0000gn/T/ipykernel_78584/2419032901.py:23: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  admin_union = gdf_sub.geometry.unary_union
/opt/anaconda3/envs/midway-debug/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: /Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/China_BuiltUp_300kCities_2020/China_BuiltUp_300kCities_2020.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


average algricultural land rent in Anhui-Anqing is 29.408967034675488 CNY/m2


## 拆分农村地租数据，匹配省市县

In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


agr_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/agr_val_2022.csv"
admin_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Raw/China_Adm_2020/China2020County.shp"
output_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/agr_val_with_admin.csv"



df = pd.read_csv(agr_path)
gdf_agr = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")


gdf_admin = gpd.read_file(admin_path).to_crs("EPSG:4326")
gdf_admin = gdf_admin.rename(columns={
    "NAME_1": "province",
    "NAME_2": "city",
    "NAME_3": "county"
})
if not gdf_admin.is_valid.all():
    gdf_admin["geometry"] = gdf_admin.buffer(0)


gdf_joined = gpd.sjoin(
    gdf_agr,
    gdf_admin[["province", "city", "county", "geometry"]],
    how="left",
    predicate="intersects"
)


n_total = len(gdf_joined)
n_matched = gdf_joined["province"].notna().sum()
match_rate = n_matched / n_total * 100

print(f"successfully matched {n_matched} out of {n_total} points.")
print(f"match rate: {match_rate:.2f}%")


gdf_joined.drop(columns=["geometry", "index_right"], inplace=True)
gdf_joined = gdf_joined.dropna(subset=["province"])
gdf_joined.to_csv(output_path, index=False)


print(f"completed! The output file is saved at {output_path}.")


successfully matched 73003 out of 157267 points.
match rate: 46.42%
completed! The output file is saved at /Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/agr_val_with_admin.csv.


In [ ]:
import pandas as pd
import os

agr_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/agr_val_with_admin.csv"
agr_output_dir = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/Agr_lp"
os.makedirs(agr_output_dir, exist_ok=True)

df = pd.read_csv(agr_path)

df["city"] = df["city"].replace(["NULL", "null", "", "NaN", "nan"], pd.NA)

def generate_prov_city(row):
    if pd.notna(row["city"]):
        return f"{row['province']}-{row['city']}"
    elif pd.notna(row["county"]):
        return f"{row['province']}-{row['county']}"
    else:
        return pd.NA

df["prov_city"] = df.apply(generate_prov_city, axis=1)


for prov_city, group in df.groupby("prov_city"):
    if pd.isna(prov_city):
        continue

    filename = f"{prov_city.strip().replace(' ', '_')}_agr_lp.csv"
    output_path = os.path.join(agr_output_dir, filename)

    group.to_csv(output_path, index=False)
    print(f"saved {len(group)} points to {output_path}")

print("completed!")



In [11]:
df.shape

(73003, 10)

In [12]:
df["prov_city"].nunique()

440

In [81]:
df.head()

,lon,lat,value,area_m2,price_tal_are,price_m2,province,city,county,prov_city
0,110.625000,42.458333,113.744060,6.348930e+07,1533700.4,0.024157,Neimenggu,Baotou,Daerhanmaominganlianhe,Neimenggu-Baotou
1,109.625000,42.208333,283.251280,6.374217e+07,3819299.2,0.059918,Neimenggu,Baotou,Daerhanmaominganlianhe,Neimenggu-Baotou
2,110.125000,42.208333,137.060730,6.374217e+07,1848097.5,0.028993,Neimenggu,Baotou,Daerhanmaominganlianhe,Neimenggu-Baotou
3,110.041667,42.125000,46.350080,6.382619e+07,624974.5,0.009792,Neimenggu,Baotou,Daerhanmaominganlianhe,Neimenggu-Baotou
4,109.541667,42.041667,74.622765,6.391007e+07,1006197.4,0.015744,Neimenggu,Baotou,Daerhanmaominganlianhe,Neimenggu-Baotou


## 匹配建成区外的点

In [80]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# === Path settings ===
input_dir = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/Agr_lp"
builtup_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/China_BuiltUp_300kCities_2020/China_BuiltUp_300kCities_2020.shp"

# === Load built-up area shapefile ===
gdf_builtup = gpd.read_file(builtup_path).to_crs("EPSG:4326")
if not gdf_builtup.is_valid.all():
    gdf_builtup["geometry"] = gdf_builtup.buffer(0)

# === Statistics ===
zero_remaining_files = 0
total_files = 0
zero_file_list = []

# === Iterate over all CSV files ===
for filename in os.listdir(input_dir):
    if not filename.endswith(".csv"):
        continue

    total_files += 1
    file_path = os.path.join(input_dir, filename)

    # Read CSV and construct GeoDataFrame
    df = pd.read_csv(file_path)
    if df.empty or "lon" not in df.columns or "lat" not in df.columns:
        print(f"Warning: File {filename} is missing required fields or is empty. Skipping.")
        continue

    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326")

    # Spatial join to check whether points fall within built-up areas
    gdf_filtered = gpd.sjoin(gdf, gdf_builtup, predicate="within", how="left")
    gdf_remaining = gdf_filtered[gdf_filtered["index_right"].isna()]  # Points not within built-up areas

    # Print remaining point count for each file
    if len(gdf_remaining) == 0:
        zero_remaining_files += 1
        zero_file_list.append(filename)

# === Summary statistics ===
zero_ratio = zero_remaining_files / total_files * 100 if total_files > 0 else 0
print(f"\nTotal number of files: {total_files}")
print(f"Number of files with 0 remaining points: {zero_remaining_files}")
print(f"Proportion: {zero_ratio:.2f}%")

# === Print list of filenames with zero remaining points ===
if zero_file_list:
    print("\nList of files with 0 remaining points:")
    for fname in zero_file_list:
        print(f"- {fname}")


/opt/anaconda3/envs/midway-debug/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: /Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/China_BuiltUp_300kCities_2020/China_BuiltUp_300kCities_2020.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(



Total number of files: 440
Number of files with 0 remaining points: 9
Proportion: 2.05%

List of files with 0 remaining points:
- Shanghai-Yangpu_agr_lp.csv
- Shanghai-Putuo_agr_lp.csv
- Beijing-Shijingshan_agr_lp.csv
- Shanghai-Changning_agr_lp.csv
- Shanghai-Huangpu_agr_lp.csv
- Shanghai-Hongkou_agr_lp.csv
- Chongqing-Dadukou_agr_lp.csv
- Shanghai-Minhang_agr_lp.csv
- Tianjin-Nankai_agr_lp.csv


## 计算平均农业用地价格

In [85]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# === Path settings ===
input_dir = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/Agr_lp"
builtup_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/China_BuiltUp_300kCities_2020/China_BuiltUp_300kCities_2020.shp"
avg_output_path = "/Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/avg_lp.csv"

gdf_builtup = gpd.read_file(builtup_path).to_crs("EPSG:4326")
if not gdf_builtup.is_valid.all():
    gdf_builtup["geometry"] = gdf_builtup.buffer(0)


results = []


for filename in os.listdir(input_dir):
    if not filename.endswith(".csv"):
        continue

    file_path = os.path.join(input_dir, filename)
    df = pd.read_csv(file_path)

    if df.empty or "lon" not in df.columns or "lat" not in df.columns or "price_m2" not in df.columns:
        continue


    prov_city = filename.replace("_agr_lp.csv", "")

    if len(df) <= 5:
        avg_price = df["price_m2"].mean() / 0.05
        results.append({"prov_city": prov_city, "land_price": avg_price})
        continue

    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326")
    gdf_filtered = gpd.sjoin(gdf, gdf_builtup, predicate="within", how="left")
    gdf_outside = gdf_filtered[gdf_filtered["index_right"].isna()]

    if gdf_outside.empty:
        continue 
    else:
        avg_price = gdf_outside["price_m2"].mean() / 0.05
        results.append({"prov_city": prov_city, "land_price": avg_price})

df_result = pd.DataFrame(results)
df_result.to_csv(avg_output_path, index=False)
print(f"Saved average land price estimates to: {avg_output_path}")


/opt/anaconda3/envs/midway-debug/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: /Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/China_BuiltUp_300kCities_2020/China_BuiltUp_300kCities_2020.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Saved average land price estimates to: /Users/yxy/UChi/Spring2025/MACS30123/Final_project/Data/Cleaned/avg_lp.csv



## Land Output Elasticity

Due to substantial missing data on key housing-related variables across many cities, it is difficult to reliably estimate the land output elasticity empirically for each city. Therefore, we adopt a unified value for the land output elasticity $\alpha$ based on estimates from the existing literature, and use it consistently to compute urban land rents.

In this study, we set $\alpha = 0.3$, following the empirical estimates reported in Table 1 of **Zhang and Jin (2012)**. The authors estimate a translog stochastic frontier production function that includes capital (K), labor (L), and land (l) as inputs. According to their results, the estimated elasticity of land—captured by the coefficient on $\ln(l)$—is **0.293**, and is statistically significant at the 5% level. This value reflects the marginal contribution of land to output across Chinese cities.

Thus, using $\alpha = 0.3$ serves as a theoretically grounded and empirically supported calibration for decomposing land rent.

